In [41]:
import pandas as pd
import pyodbc
import os
import time
from typing import Tuple

In [43]:

class DatabaseConnection:
    """ Class to handle database connections. """

    def __init__(self, server: str, database: str):
        self.server = server
        self.database = database
        self.connection_string = f'DRIVER={{SQL Server}};SERVER={self.server};DATABASE={self.database};Trusted_Connection=yes;'
        self.connection = None

    def connect(self):
        """ Establish a connection to the database. """
        try:
            self.connection = pyodbc.connect(self.connection_string)
            print("Connected successfully to the database.")
        except pyodbc.Error as e:
            print("Connection error:", e)
            raise

    def close(self):
        """ Close the database connection. """
        if self.connection:
            self.connection.close()
            print("Connection closed.")

    def is_connected(self):
        """ Check if the connection is active. """
        return self.connection is not None


In [45]:

class DataPopulator:
    """ Class to handle loading data from CSV and populating the database. """

    def __init__(self, db_connection: DatabaseConnection):
        self.db_connection = db_connection

    def load_data_from_csv(self, file_path: str) -> pd.DataFrame:
        """ Load data from a CSV file into a DataFrame. """
        if not os.path.isfile(file_path):
            raise FileNotFoundError(f"'{file_path}' does not exist.")
        df = pd.read_csv(file_path)
        if df.empty:
            raise ValueError("The CSV file is empty.")
        return df

    def validate_data(self, df: pd.DataFrame):
        """ Validate the DataFrame for required fields and correct data types. """
        required_columns = [
            'Rating', 'Author', 'Author Location', 'Review Date',
            'Review Title', 'Review', 'Type of Traveller', 'Seat Type',
            'Route', 'Date Flown', 'Seat Comfort', 'Cabin Staff Service',
            'Food & Beverages', 'Inflight Entertainment', 
            'Ground Service', 'Value for Money', 'Recommended Service'
        ]
        
        # Check for missing required columns
        for column in required_columns:
            if column not in df.columns:
                raise ValueError(f"Missing required column: {column}")
    
        # Validate Rating
        if df['Rating'].isnull().any():
            raise ValueError("Rating has missing values.")
        if not pd.api.types.is_numeric_dtype(df['Rating']):
            raise TypeError("Rating must be numeric.")
        
        # Validate Author
        if df['Author'].isnull().any():
            raise ValueError("Author has missing values.")
        if not pd.api.types.is_string_dtype(df['Author']):
            raise TypeError("Author must be a string.")

        # Validate Recommended Service
        if df['Recommended Service'].isnull().any():
            raise ValueError("Recommended Service has a missing values.")
        if not pd.api.types.is_string_dtype(df['Recommended Service']):
            raise TypeError("Recommended Service must be a string or char.")
            
        # Validate Review Date
        if df['Review Date'].isnull().any():
            raise ValueError("Review Date has missing values.")
        try:
            pd.to_datetime(df['Review Date'])
        except Exception:
            raise ValueError("Review Date must be in a valid date format.")
        
        # Validate Other Date Fields
        if df['Date Flown'].isnull().any():
            raise ValueError("Date Flown has missing values.")
        try:
            pd.to_datetime(df['Date Flown'])
        except Exception:
            raise ValueError("Date Flown must be in a valid date format.")
    
        # Validate Additional Ratings
        rating_columns = ['Seat Comfort', 'Cabin Staff Service', 'Food & Beverages', 
                          'Inflight Entertainment', 'Ground Service', 
                          'Value for Money']
        
        for column in rating_columns:
            if column in df.columns and df[column].isnull().any():
                raise ValueError(f"{column} has missing values.")
            
            if column in df.columns and not pd.api.types.is_numeric_dtype(df[column]):
                raise TypeError(f"{column} must be numeric.")
        
        # Validate Heading and Review
        for column in ['Review Title', 'Review']:
            if df[column].isnull().any():
                raise ValueError(f"{column} has missing values.")
            if not pd.api.types.is_string_dtype(df[column]):
                raise TypeError(f"{column} must be a string.")
    
        # Validate Type of Traveller and Seat Type
        for column in ['Type of Traveller', 'Seat Type']:
            if df[column].isnull().any():
                raise ValueError(f"{column} has missing values.")
            if not pd.api.types.is_string_dtype(df[column]):
                raise TypeError(f"{column} must be a string.")

    def insert_data_to_db(self, df: pd.DataFrame):
        """ Insert DataFrame data into the database. """
        self.validate_data(df)
        
        insert_query = """
            INSERT INTO  Reviews (
                Rating, Author, [Author Location], [Review Date], [Review Title], Review,
                [Type Of Traveller], [Seat Type], Route, [Date Flown],
                [Seat Comfort], [Cabin Staff Service], [Food and Beverages],
                [Inflight Entertainment], [Ground Service], [Value For Money],
                [Recommended Service]
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        cursor = self.db_connection.connection.cursor()
        row_count = 0  
        
        for index, row in df.iterrows():
            params = tuple(row)
            cursor.execute(insert_query, params)
            row_count += 1

        self.db_connection.connection.commit()
        print(f"Data inserted successfully. Total rows inserted: {row_count}.")



In [48]:

def main_check_connection(db_server: str, db_name: str):
    """ Check database connection. """
    db_connection = DatabaseConnection(db_server, db_name)
    db_connection.connect()
    db_connection.close()


def main_load_data(db_server: str, db_name: str, csv_file: str):
    """ Load data from CSV into the database. """
    db_connection = DatabaseConnection(db_server, db_name)
    db_connection.connect()
    
    data_populator = DataPopulator(db_connection)
    
    try:
        df = data_populator.load_data_from_csv(csv_file)
        data_populator.insert_data_to_db(df)
    except (FileNotFoundError, ValueError, TypeError) as e:
        print(e)
    finally:
        db_connection.close()


if __name__ == "__main__":
    SERVER = 'localhost'
    DATABASE = 'BritishAirwaysDB'
    CSV_FILE = 'scraped_reviews.csv'

    
    main_check_connection(SERVER, DATABASE)
    main_load_data(SERVER, DATABASE, CSV_FILE)

Connected successfully to the database.
Connection closed.
Connected successfully to the database.
Data inserted successfully. Total rows inserted: 428.
Connection closed.
